In [1]:
from lib.solver_tools.solver_client import SolverClient
from lib.solver_tools import Task
from lib.solver_tools.utils import make_task_comparison_link
import json
from copy import deepcopy
from lib.common_tools.read_token import read_solver_apikey
import os

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import vrp_algorithms_lib.common_tools.file_utils as file_utils

In [3]:
my_apikey = read_solver_apikey()
solver_client = SolverClient(my_apikey)

In [16]:
# request_path = '../test_data/inputs/medium_test_1/request.json'
# task_id = '34000a30-948d94b2-8665fb47-a7eb3b85'

# request_path = '../test_data/inputs/large_test_1/request.json'
# task_id = '88060738-3eb5db87-9f6a7288-6fd4c3d2'

request_path = '../test_data/inputs/large_test_2/request.json'
task_id = 'f392f33d-1bab419a-24f43d12-acf0c715'

In [17]:
initial_request = solver_client.get_request(task_id)

In [24]:
def transform_request(initial_request):
    assert len(initial_request['depots']) == 1
    
    locations = [
        {'id': f'location {i+1}', 'depot_id': 'depot 1', 'point': l['point'], 'time_window': '08:00:00-22:00:00'} for i, l in enumerate(initial_request['locations'])  # could take l['time_window'] for time_window
    ]
    
    vehicles = [
        {'id': f'courier {i+1}', 'return_to_depot': False} for i, v in enumerate(initial_request['vehicles'])
    ]
    
    depots = [
        {'id': f'depot {i+1}', 'point': d['point'], 'time_window': '00:00:00-2.00:00:00'} for i, d in enumerate(initial_request['depots'])
    ]
    
    options = {
        'time_zone': 3
    }
    
    new_request = {
        'locations': locations,
        'vehicles': vehicles,
        'depots': depots,
        'options': options
    }
    
    return new_request

In [25]:
transformed_request = transform_request(initial_request)

In [26]:
file_utils.save_json(transformed_request, request_path)